In [ ]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math 

# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# for문으로 크롤링 

In [ ]:
# 검색어 입력
search = '삼성전자'

# 크롬 옵션
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=1000,1200") # window-size -> 기본 : 1920,1080

import chromedriver_autoinstaller

chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(0.01)
driver.get("https://www.naver.com") # -> 네이버 크롬 창이 뜬다.

# 네이버 검색어 입력 후 검색
element = driver.find_element("name", "query") 
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(search)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(0.01)

In [ ]:
# 뉴스 클릭
driver.find_element("link text", "뉴스").click() 
time.sleep(0.01)

In [ ]:
# '오래된 순' 클릭
driver.find_element("link text", "오래된순").click()  
time.sleep(0.01)

In [ ]:
# 뉴스 데이터 수집
# news_list = []
# date_list = []
# title_list = []

for i in range(400):    

    # 언론사
    newses = driver.find_elements('css selector', 'a.info.press')
    temp_list = []
    for news in newses:
        temp_list.append(news.text)
        temp_list = [x for x in temp_list ]
    news_list = news_list + temp_list 
    
    # 날짜
    dates = driver.find_elements('css selector', 'span.info')
    temp_list = []
    for date in dates:
        temp_list.append(date.text)
        temp_list = [x for x in temp_list ]
    date_list = date_list + temp_list 
    
    # 제목
    titles = driver.find_elements('css selector', 'a.news_tit')
    temp_list = []
    for title in titles:
        temp_list.append(title.text)
        temp_list = [x for x in temp_list ]
    title_list = title_list + temp_list 
    
    # 다음페이지 클릭
    driver.find_element('css selector', 'a.btn_next').click()
    time.sleep(0.001)

print(len(title_list))
print(news_list)
print(date_list)
print(title_list)

In [ ]:
# df로 만들기

df = pd.DataFrame(list(zip(news_list, date_list, title_list)), columns = ['언론사','날짜', '제목'])

df

# csv로 뽑기
# df.to_csv('.csv')